In [4]:
import itertools
import gurobipy as gpy
import numpy as np

In [15]:
n = 20
d = np.genfromtxt(f'dist_{n}.txt')
locs = range(n)
addedConstr = True

'''Construct model and optimize'''
m = gpy.Model('mst')
x = [m.addVar(vtype=gpy.GRB.BINARY,name=f'x_{i}_{j}') for i in range(n) for j in range(i+1,n)]
m.update()
m.setObjective((x*d).sum(), gpy.GRB.MINIMIZE) 
m.addLConstr(gpy.quicksum(x) == n - 1)
m.update()
m.optimize()

'''Print solution'''
# for v in m.getVars():
#     print(f'{v.varName}: {v.x}')

'''Search for subtours'''
'''Loop until no new constraints added'''
count = 0
while addedConstr:
    count += 1
    addedConstr = False

    ''' Consider all subsets of locations for all possible number of subset sizes m '''
    for mm in range(2, n):
        ''' Construct all m-location combinations of the location set '''
        subsets = itertools.combinations(locs, mm)

        ''' construct constraints for all of the combinations '''
        for subset in subsets:
            num_links = sum([m.getVarByName(f'x_{i}_{j}').x for i,j in itertools.combinations(subset,2)])
            if num_links >= len(subset):
                ''' Construct subtour constraint for this combination '''
                addedConstr = True
                m.addConstr(gpy.quicksum(m.getVarByName(f'x_{i}_{j}') for i,j in itertools.combinations(subset,2)) <= len(subset) - 1)
    
    '''Update model after adding new constraints'''
    m.update()
    m.optimize()

for v in m.getVars():
    if v.x > 0.5:  
        print(f'{v.varName}: {v.x}')
print(count)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1 rows, 190 columns and 190 nonzeros
Model fingerprint: 0x85c0d2dd
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 821.5000000
Presolve removed 1 rows and 190 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 291.5 821.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.915000000000e+02, best bound 2.915000000000e+02, gap 0.0000%
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (w